# Geo Location Distribution

#### In this project, I would like to explore the market and its customer in geological approach.

### Import_Useful_Libraries

In [52]:
import numpy as np
import pandas as pd
from os import path
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.express as px 
import plotly.graph_objects as go
import geopy
import folium
from geopy.geocoders import Nominatim, GoogleV3

### Read_Useful_Dataset

In [53]:
geo=pd.read_csv('C:\\Users\\imnab\\Downloads\\OVO_DS-ecommerce-brazilian\\geolocation_olist_public_dataset.csv')
order=pd.read_csv('C:\\Users\\imnab\\Downloads\\OVO_DS-ecommerce-brazilian\\olist_public_dataset_v2.csv')

### Data_Preprocessing

#### First, I am gonna explore the datasets.

In [54]:
geo.head()

,zip_code_prefix,city,state,lat,lng
0,10,sao paulo,sp,-23.547807,-46.636122
1,10,sao paulo,sp,-23.546081,-46.644820
2,10,sao paulo,sp,-23.540984,-46.642569
3,10,sao paulo,sp,-23.546758,-46.645771
4,10,sao paulo,sp,-23.546362,-46.643073


In [55]:
geo.shape

(323016, 5)

In [56]:
geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 323016 entries, 0 to 323015
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   zip_code_prefix  323016 non-null  int64  
 1   city             323016 non-null  object 
 2   state            323016 non-null  object 
 3   lat              323016 non-null  float64
 4   lng              323016 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 12.3+ MB


In [57]:
#Before going any further, it seems that geo dataset contain several duplicates. Therefore, we gonna need to drop it first.
geo = geo.drop_duplicates(subset=None, keep='first', inplace=False)

In [58]:
geo.shape

(275854, 5)

In [59]:
geo.describe()

,zip_code_prefix,lat,lng
count,275854.000000,275854.000000,275854.000000
mean,479.211456,-20.009886,-46.499468
std,307.911735,7.019831,5.109505
min,10.000000,-36.605374,-72.927296
25%,193.000000,-23.643879,-49.933594
50%,412.000000,-22.345569,-46.703234
75%,785.000000,-16.625177,-43.368705
max,999.000000,42.439286,13.820214


In [60]:
order.head()

,order_id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,...,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,b95df3cef5297e79ef709ba256518f6f,delivered,349.90,13.84,1,1,2017-01-31 17:19:01.000000,2017-02-01 02:41:21.549551,2017-03-15 00:00:00.000000,2017-02-06 11:04:24.154259,...,51,625,1,6cdd53843498f92890544667809f1595,b95df3cef5297e79ef709ba256518f6f,5,NaN,NaN,2017-02-07 00:00:00.000000,2017-02-09 02:37:37+00:00
1,59af46052a799e80e2f0c665c587731d,delivered,15.00,15.10,1,1,2017-09-09 19:52:54.000000,2017-09-10 20:03:31.535281,2017-10-02 00:00:00.000000,2017-09-13 20:17:41.296915,...,44,1428,2,ae5cad88462eb7b7b61401e31c45618e,59af46052a799e80e2f0c665c587731d,5,NaN,entrega em 2 dias produto c boa qualidade otim...,2017-09-14 00:00:00.000000,2017-09-15 03:43:47+00:00
2,a3e6136894621db402a772c6bc72a12a,delivered,238.90,18.00,1,1,2017-01-30 17:00:09.000000,2017-01-30 17:31:25.438253,2017-03-07 00:00:00.000000,2017-02-06 15:43:04.758566,...,55,637,1,0c9ff9d8ed9b9bdd825487b3a66e05f5,a3e6136894621db402a772c6bc72a12a,5,NaN,produto veio antes do prazo informado muito ob...,2017-02-07 00:00:00.000000,2017-02-10 14:18:53+00:00
3,b675ea5a618922f6e679e30531b8957b,delivered,29.99,18.23,1,1,2018-03-11 18:18:36.000000,2018-03-11 18:30:37.931962,2018-04-03 00:00:00.000000,2018-04-03 20:36:43.778451,...,55,617,1,ad0a798e7941f3a5a2fb8139cb62ad78,b675ea5a618922f6e679e30531b8957b,4,NaN,NaN,2018-04-04 00:00:00.000000,2018-04-05 02:52:31+00:00
4,195a8be6794c487fe6cfbb97b7c61902,delivered,295.99,47.65,1,1,2017-04-20 08:01:08.000000,2017-04-25 08:05:40.405383,2017-05-24 00:00:00.000000,2017-05-04 18:47:45.721758,...,49,558,2,eaf2046d4c87809247a30050ea13df03,195a8be6794c487fe6cfbb97b7c61902,5,NaN,NaN,2017-05-05 00:00:00.000000,2017-05-08 15:20:18+00:00


In [61]:
order.shape

(100000, 25)

In [62]:
order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 25 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       100000 non-null  object 
 1   order_status                   100000 non-null  object 
 2   order_products_value           100000 non-null  float64
 3   order_freight_value            100000 non-null  float64
 4   order_items_qty                100000 non-null  int64  
 5   order_sellers_qty              100000 non-null  int64  
 6   order_purchase_timestamp       100000 non-null  object 
 7   order_aproved_at               99982 non-null   object 
 8   order_estimated_delivery_date  100000 non-null  object 
 9   order_delivered_customer_date  97595 non-null   object 
 10  customer_id                    100000 non-null  object 
 11  customer_city                  100000 non-null  object 
 12  customer_state                 

In [63]:
#Before going any further, it seems that geo dataset contain several duplicates. Therefore, we gonna need to drop it first.
order = order.drop_duplicates(subset=None, keep='first', inplace=False)

In [64]:
order.shape

(100000, 25)

In [65]:
order.describe()

,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,review_score
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.0000,100000.000000
mean,129.191704,21.739202,1.100090,1.141690,353.768400,48.839630,779.020710,2.2781,4.054110
std,194.863060,20.088368,0.459189,0.708829,297.682699,10.139335,665.387848,1.7495,1.363591
min,2.000000,0.000000,1.000000,1.000000,10.000000,5.000000,8.000000,1.0000,1.000000
25%,46.990000,13.470000,1.000000,1.000000,116.000000,42.000000,340.000000,1.0000,4.000000
50%,83.650000,16.790000,1.000000,1.000000,250.000000,52.000000,591.000000,2.0000,5.000000
75%,145.900000,23.010000,1.000000,1.000000,590.000000,57.000000,978.000000,3.0000,5.000000
max,13440.000000,1562.100000,20.000000,30.000000,999.000000,72.000000,3992.000000,20.0000,5.000000


## 1) Market_Distribution

#### Before we analyze the distribution of market location, we need to process the geo dataset. As we have seen from the dataset above, one zip_code_prefix may contain several different latitudes and longitudes.Since there is big amount of data, it would be better to group each zip code and define its langitude and longitude based on each  zip code center as shown below.The count column define number of different position for each zip code.


In [66]:
centroid = geo.groupby('zip_code_prefix').agg({
    'lat': 'median',
    'lng': 'median',
    'city': pd.Series.mode
}).reset_index()
centroid['count'] = geo.groupby('zip_code_prefix').size().reset_index(
    name='counts')['counts']

In [67]:
centroid.head()

,zip_code_prefix,lat,lng,city,count
0,10,-23.545520,-46.636225,sao paulo,323
1,11,-23.528066,-46.649218,sao paulo,316
2,12,-23.540822,-46.654373,sao paulo,419
3,13,-23.556829,-46.648644,sao paulo,348
4,14,-23.567885,-46.663207,sao paulo,414


#### The following map are the distribution of the market at Brazil:

In [68]:
token = 'pk.eyJ1IjoiZmFhdGhpcjk4IiwiYSI6ImNrMGVxMTl5dzBiZHYzYm1wMm15b2RhcXQifQ.InHG7-SGeNMtnnfkvIVCRg'
fig = px.scatter_mapbox(centroid,
                        lat="lat",
                        lon="lng",
                        size="count",
                        color="count",
                        size_max=8,
                        color_continuous_scale=px.colors.sequential.Redor,
                        zoom=2.7)
fig.update_layout(mapbox_style="outdoors", mapbox_accesstoken=token)
fig.show()

#### As we can see, the market distributed highly at south Brazil approximately around lat: -22 and lng: -46. Therefore, I suggest for the market distribution to be balanced. Moreover, if the company interested in expand the market to international, it is highly recommended to expand the market at north and west Brazil since there is only a few market around to focus on company's branding.

## 2) Revenue_Dustribution

#### To explore the distribution of product revenue,we need to join both dataset based on order id column because position for each order are contained on the geo dataset

In [69]:
order.head()

,order_id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,...,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,b95df3cef5297e79ef709ba256518f6f,delivered,349.90,13.84,1,1,2017-01-31 17:19:01.000000,2017-02-01 02:41:21.549551,2017-03-15 00:00:00.000000,2017-02-06 11:04:24.154259,...,51,625,1,6cdd53843498f92890544667809f1595,b95df3cef5297e79ef709ba256518f6f,5,NaN,NaN,2017-02-07 00:00:00.000000,2017-02-09 02:37:37+00:00
1,59af46052a799e80e2f0c665c587731d,delivered,15.00,15.10,1,1,2017-09-09 19:52:54.000000,2017-09-10 20:03:31.535281,2017-10-02 00:00:00.000000,2017-09-13 20:17:41.296915,...,44,1428,2,ae5cad88462eb7b7b61401e31c45618e,59af46052a799e80e2f0c665c587731d,5,NaN,entrega em 2 dias produto c boa qualidade otim...,2017-09-14 00:00:00.000000,2017-09-15 03:43:47+00:00
2,a3e6136894621db402a772c6bc72a12a,delivered,238.90,18.00,1,1,2017-01-30 17:00:09.000000,2017-01-30 17:31:25.438253,2017-03-07 00:00:00.000000,2017-02-06 15:43:04.758566,...,55,637,1,0c9ff9d8ed9b9bdd825487b3a66e05f5,a3e6136894621db402a772c6bc72a12a,5,NaN,produto veio antes do prazo informado muito ob...,2017-02-07 00:00:00.000000,2017-02-10 14:18:53+00:00
3,b675ea5a618922f6e679e30531b8957b,delivered,29.99,18.23,1,1,2018-03-11 18:18:36.000000,2018-03-11 18:30:37.931962,2018-04-03 00:00:00.000000,2018-04-03 20:36:43.778451,...,55,617,1,ad0a798e7941f3a5a2fb8139cb62ad78,b675ea5a618922f6e679e30531b8957b,4,NaN,NaN,2018-04-04 00:00:00.000000,2018-04-05 02:52:31+00:00
4,195a8be6794c487fe6cfbb97b7c61902,delivered,295.99,47.65,1,1,2017-04-20 08:01:08.000000,2017-04-25 08:05:40.405383,2017-05-24 00:00:00.000000,2017-05-04 18:47:45.721758,...,49,558,2,eaf2046d4c87809247a30050ea13df03,195a8be6794c487fe6cfbb97b7c61902,5,NaN,NaN,2017-05-05 00:00:00.000000,2017-05-08 15:20:18+00:00


In [70]:
geo_order = pd.merge(centroid,
                     order,
                     how='right',
                     left_on='zip_code_prefix',
                     right_on='customer_zip_code_prefix')

#### Same as the previous part, the dataset is grouped by customer zip code and for each group contain the sum of value ordered products.

In [71]:
geo_rev = geo_order.groupby('customer_zip_code_prefix').agg({
    'lat':
    'median',
    'lng':
    'median',
    'order_products_value':
    'sum'
}).reset_index()

#### The following map are the distribution of the purchased procuts at Brazil:

In [72]:
fig = go.Figure(
    go.Densitymapbox(lat=geo_rev.lat,
                     lon=geo_rev.lng,
                     z=geo_rev.order_products_value,
                     radius=10))
fig.update_layout(mapbox_style="stamen-terrain",
                  mapbox_center_lon=-50,
                  mapbox_center_lat=-16,
                  mapbox_zoom=2.7)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

#### As shown the map above, we can conclude there are three main hotspots which located at approximately:

#### lat:-24, lng:-46
#### lat:-22, lng:-43
#### lat:-19, lng:-43
#### These three places are the market with highest revenue. To raise company's revenue, it is highly recommended to add more market around these three hotspots since there's a lot of customer gladly spending their money there.

## 3) Customer_Distribution

#### The same method is used to find the central of each zip code, additonally with defining number of customer id as count.

In [73]:
geo_order = geo_order.dropna(subset=['zip_code_prefix'])
geo_cust = geo_order.groupby('customer_zip_code_prefix').agg({
    'lat':
    'median',
    'lng':
    'median',
    'customer_id':
    'count'
}).reset_index()

#### The following map are the distribution of the customer products at Brazil:

In [74]:
fig = go.Figure(
    go.Densitymapbox(lat=geo_cust.lat,
                     lon=geo_cust.lng,
                     z=geo_cust.customer_id,
                     radius=10))
fig.update_layout(mapbox_style="stamen-terrain",
                  mapbox_center_lon=-50,
                  mapbox_center_lat=-16,
                  mapbox_zoom=2.7)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show()

#### As shown above, there are three main hotspots with highest number of customer. These hotspots are located at approximately:

#### lat:-23, lng:-46
#### lat: -22, lng:-43
#### lat: -19, lng:-43
#### Note that the location of these hotspots approximately near with the previous hotspots, so we can assume that these hotspots are simmiliarly the same. With higher number of customer, these places would great place to expand your market because of customer's interest with this market.

# Conclusion

#### In this geological approach, there are a few things i would like to recommend:

#### a) For branding expansion, it is best for the company to expand the market at the west and north Brazil. However, with low income from these location, it would be better to build only a few market for each city or area since the interest of the customer is currently low.
#### b) For increase in revenue, it is best for the company to expand the market at these three hotspots:
####    1) lat:-23, lng:-46
####    2) lat: -22, lng:-43
####    3) lat: -19, lng:-43
#### Since these hotspots have highest number of customer and revenue, it shows that the customer around these hotspots interested with the market.